In [1]:
import torch 
from egg.zoo.pop.utils import get_common_opts, metadata_opener, load_from_checkpoint
from egg.zoo.pop.games import build_game
import hub
from torchvision import transforms

#from egg.zoo.pop.data import ImageTransformation
import numpy as np

/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../../../../../experiments/cont_fuller_pop/199711/final.tar"
metadata_path = "../../../../../experiments/cont_fuller_pop/199711/wandb/latest-run/files/wandb-metadata.json"

In [3]:
opts = None
with open(metadata_path) as f:
    opts = get_common_opts(metadata_opener(f, data_type="wandb", verbose=True))

In [4]:
pop_game = build_game(opts)
load_from_checkpoint(pop_game, model_path)
senders = pop_game.agents_loss_sampler.senders
for sender in senders:
    sender.eval()
    # TODO : remove gradients

initialize module vgg11


/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize module vit
initialize module resnet152


/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize module inception


/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize module swin


/homedtcl/mmahaut/.conda/envs/omelette/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


initialize module dino
initialize module vgg11
initialize module vit
initialize module resnet152
initialize module inception
initialize module swin
initialize module dino
['vgg11', 'vit', 'resnet152', 'inception', 'swin', 'dino']
['vgg11', 'vit', 'resnet152', 'inception', 'swin', 'dino']
# loading trainer state from ../../../../../experiments/cont_fuller_pop/199711/final.tar


In [5]:
ds = hub.load("hub://activeloop/places205")

hub://activeloop/places205 loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/places205


In [6]:
class ImageTransformation:
    """
    resize and normalize
    """
    def __init__(self, size: int):
        transformations = [
                transforms.Resize(size=(size, size)),
            ]
        transformations.append(
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            )
        self.transform = transforms.Compose(transformations)

    def __call__(self, x):
        return self.transform(x)

In [60]:
size = 384
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(int(3/x.shape[0]), 1, 1)),
    transforms.Resize(size=(size, size)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

def collate_fn(batch):
    return (
        torch.stack([x["images"] for x in batch], dim=0),
        torch.stack([torch.Tensor(x["labels"]).long() for x in batch], dim=0),
        torch.stack([torch.Tensor(x["index"]) for x in batch], dim=0),
    )
dataloader = ds.pytorch(num_workers = 0, shuffle = True, batch_size= 128, collate_fn=collate_fn, transform={'images':transformations,'labels':None, 'index':None})


In [38]:
class LinearClassifier(torch.nn.Module):
  def __init__(self, input_dim=2, output_dim=3):
    super(LinearClassifier, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    x = self.linear(x)
    return x

criterion = torch.nn.CrossEntropyLoss()

classifiers = [LinearClassifier(16, 245).to("cuda")] * len(senders) # keeping all classifiers on cpu
classifiers.append(LinearClassifier(16, 245).to("cuda")) # <-shuffled-classifier : input a random message from one of the senders

optimizers = [torch.optim.Adam(classifier.parameters(), lr=0.01) for classifier in classifiers]

In [ ]:
device="cuda"

def forward_backward(model_idx, input_images, labels):
    # everyone goes to selected device
    senders[model_idx].to(device)
    
    message = senders[model_idx](input_images)
    output = classifiers[model_idx](message)

    loss = criterion(output, labels.view(-1))
    loss.backward()

    optimizers[i].step()
    optimizers[i].zero_grad()
    
    senders[model_idx].to("cpu")

    return message, output, loss

for batch_idx, batch in enumerate(dataloader):
    _rand_sender = torch.randint(0, len(senders), (1,)).item() # chosing the shuffled input for shuffled-classifier
    images, labels, _ = batch
    images = images.to(device)
    labels = labels.to(device)
    
    for i in range(len(senders) - 1):
        _, output, loss = forward_backward(i, images, labels)
        if batch_idx % 1000 == 0 :
            print(f"acc_{i} : ", (labels.to(device) == output.argmax(dim=1)).float().mean())
        if i == _rand_sender:
            _, output, loss = forward_backward(-1, images, labels)
            if batch_idx % 1000 == 0 :
                print("acc_shuffled : ", (labels == output.argmax(dim=1)).float().mean())
    
    

acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0

acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_shuffled :  t

acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0

acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  t

acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  t

acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
acc_0 :  tensor(0., device='cuda:0')
acc_1 :  tensor(0., device='cuda:0')
acc_shuffled :  tensor(0., device='cuda:0')
acc_2 :  tensor(0., device='cuda:0')
acc_3 :  tensor(0., device='cuda:0')
acc_4 :  tensor(0., device='cuda:0')
